In [1]:
import pyaudio
import wave
import whisper
import os
from fpdf import FPDF

In [2]:
# Configure recording
CHUNK_DURATION = 10
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
CHUNK_SIZE = 1024
TEMP_FILE = "current_chunk.wav"
OUTPUT_PDF = "Meeting_ZOOM.pdf"

In [3]:
#initialize the whisper 
print ("Loading Whisper model...")
model = whisper.load_model("base")
pdf = FPDF()
pdf.add_page()

Loading Whisper model...


In [4]:
try:
    pdf.add_font('DejaVu', '', '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf', uni=True)
    pdf.set_font('DejaVu', size=12)
except:
    # Fallback if the path above is different on your ThinkPad
    pdf.set_font("Arial", size=12) 
    print("Warning: Using Arial. Special characters might still cause errors.")


/tmp/ipykernel_12129/2929088518.py:2: DeprecationWarning: "uni" parameter is deprecated since v2.5.1 and will be removed in a future release
  pdf.add_font('DejaVu', '', '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf', uni=True)


In [5]:
audio_interface = pyaudio.PyAudio()

ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_route.c:877:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:877:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:877:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:877:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'


In [6]:
def record_30_second_chunk():
    #record audio for 20 seconds --- specified by CHUNK_DURATION
    stream = audio_interface.open(format=FORMAT,
                                  channels=CHANNELS,
                                  rate=RATE,
                                  input=True,
                                  frames_per_buffer=CHUNK_SIZE)
    frames = []
    for _ in range(0, int(RATE / CHUNK_SIZE * CHUNK_DURATION)):
        data = stream.read(CHUNK_SIZE)
        frames.append(data)

    stream.stop_stream()
    stream.close()
    
    wf = wave.open(TEMP_FILE, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(audio_interface.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

In [7]:
print ("Recording... Press Ctrl+C to stop.")

try:
    while True:
        record_30_second_chunk()
        result = model.transcribe(TEMP_FILE, fp16=False)
        text = result['text'].strip()

        if text:
            print(f"Transcribed: {text}")
            pdf.multi_cell(0, 10, text)
            pdf.ln()
except KeyboardInterrupt:
    print("Recording stopped. Saving PDF...")
finally:
    pdf.output(OUTPUT_PDF)
    audio_interface.terminate()


    if os.path.exists(TEMP_FILE):
        os.remove(TEMP_FILE)    
    print(f"PDF saved as {OUTPUT_PDF}")

Recording... Press Ctrl+C to stop.
Transcribed: Слышь, я не могу. Я не могу. Я не могу. Я не могу. Я не могу.
Transcribed: like I'm just selling those hot book every now and then. So why is it transition, why is it just as much as it sounds today?
Transcribed: this part is his first fish nope alright
Transcribed: It's not just for us to be explorers or rich. We can support this institution. But also when we come up with this kind of research and findings that we've found.
Transcribed: and the law wishes will really want to support such kind of community. But also, we realize that in heavy reliance on diamonds, we are...
Transcribed: And therefore we felt that we could sign in the amount of sign that we have if we are able to tap into that potential and we see how it can be utilized in...
Transcribed: Finally we are accepting doing this. It was necessary for us to have a pick-up of assessment, collection, and community.
Transcribed: …and people that are coming from different communities